In [1]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [12]:
!pip install pinecone-client transformers

  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.6 MB/s eta 0:00:00


In [13]:
!pip install "pinecone-client[grpc]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [16]:
pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
import os
from groq import Groq
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer,  AutoModelForCausalLM

In [5]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [18]:
# Initialize Pinecone
pc = Pinecone(api_key='64396515-041d-4b41-adbc-146b4c648311')

# Access the index
index = pc.Index("ai-doctor")

# Load the tokenizer and model for embeddings
tokenizer_emb = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model_emb = SentenceTransformer("neuml/pubmedbert-base-embeddings")

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
import torch
import base64
import zlib

def decode_and_decompress_metadata(encoded_metadata):
    # Decode the Base64 string to compressed bytes
    compressed_metadata = base64.b64decode(encoded_metadata)
    # Decompress the bytes
    metadata = zlib.decompress(compressed_metadata).decode('utf-8')
    return metadata

def Retrieve(text, tokenizer=tokenizer_emb, model=model_emb, index=index, top_k=1):
    # Generate the embedding for the input text
    embedding = model_emb.encode(text).tolist()

    # Retrieve the top-k results from Pinecone
    query_response = index.query(vector=embedding, top_k=1, include_metadata=True, include_values=True)

    # Decode and decompress metadata for each result
    # results = []
    for match in query_response['matches']:
        decoded_metadata = decode_and_decompress_metadata(match['metadata']['patient_doctor_dialogue'])

        result = " ".join([str(md) for md in decoded_metadata])
        # results.append({
            # 'id': match['id'],
            # 'score': match['score'],
            # 'metadata': decoded_metadata,
            # 'values': match.get('values')
        # })

    return result

In [218]:
def generate_response_with_groq(query):
        context=Retrieve(query)
        prompt_template = """You are an doctor bot whose job is to provide diagnoses according to user's mentioned symptoms. Keep it 2-3 lines long only.

        Question: {query}

        context: {context}

         Don’t give answers out of context and irrelevant to medical and dont give justification of it. Just say that you dont know.
        answer: Based on the provided information, the possible diagnosis is with some information about it.
        """
        prompt = prompt_template.format(context=context, query=query)
        messages=[

                    {"role": "system", "content": prompt},
                    {"role": "user", "content": query}

            ]
        chat_completion = client.chat.completions.create(
            messages = messages,
            model="llama3-8b-8192",
            temperature=0,
            max_tokens=1000
        )
        return chat_completion.choices[0].message.content

In [219]:
#testing
generate_response_with_groq(
    "I feel dizzy and restless and I can't sleep well. I have trouble breathing."
)

'Based on your symptoms, I suspect you may be experiencing symptoms of Obstructive Sleep Apnea (OSA). OSA is a condition where the airway is partially blocked during sleep, causing shallow or paused breathing, leading to poor sleep quality, dizziness, and restlessness.'

In [220]:
#Irrelevant question
generate_response_with_groq(
    "what is astrology."
)

"I'm a doctor bot, and I don't diagnose or provide information on non-medical topics like astrology. I'm here to help with medical-related questions and provide diagnoses based on symptoms. Astrology is not a medical concept, and I don't have any information about it."

In [222]:
#Few symptoms mentioned
generate_response_with_groq(
    "I feel cold and have nausea."
)

'Based on your symptoms of feeling cold and nausea, I suspect you may be experiencing a hypothermic episode, possibly triggered by your previous episode of nausea and hot sweats.'

In [224]:
#Typos
generate_response_with_groq(
    "whap do i doooo. i feel like i faintp"
)

'I\'m not sure what you mean by "whap do i doooo" and "i feel like i faintp", but based on your previous message, it seems like you\'re experiencing some unusual sensations in your body. You mentioned feeling like you\'re going to faint, which could be related to a variety of conditions such as vasovagal syncope, hyperventilation, or even a panic attack. I would recommend consulting a doctor to rule out any underlying medical conditions and to get a proper diagnosis and treatment plan.'